# Chicago Dataset - Analysing data with SQL and Python

### Understand the datasets

To complete the assignment problems in this notebook you will be using three datasets that are available on the city of Chicago's Data Portal:

Socioeconomic Indicators in Chicago
Chicago Public Schools
Chicago Crime Data

1. Socioeconomic Indicators in Chicago
This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:

https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2

2. Chicago Public Schools
This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:

https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t

3. Chicago Crime Data
This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:

https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2

### Store the datasets in database tables
To analyze the data using SQL, it first needs to be loaded into SQLite DB. We will create three tables in as under:

1. CENSUS_DATA
2. CHICAGO_PUBLIC_SCHOOLS
3. CHICAGO_CRIME_DATA






Load the pandas and sqlite3 libraries and establish a connection to FinalDB.db

In [1]:
import pandas
import csv, sqlite3

con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

Load the SQL magic module

In [2]:
%load_ext sql

Use Pandas to load the data available in the links above to dataframes. Use these dataframes to load data on to the database FinalDB.db as required tables.

In [3]:
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists= 'replace', index=False, method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists='replace', index=False, method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

533

Establish a connection between SQL magic module and the database FinalDB.db

In [4]:
%sql sqlite:///FinalDB.db

In [5]:
# List of all tables in the databases
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///FinalDB.db
Done.


name
CENSUS_DATA
CHICAGO_PUBLIC_SCHOOLS
CHICAGO_CRIME_DATA


### Problems
Now write and execute SQL queries to solve assignment problems

### Problem 1
Find the total number of crimes recorded in the CRIME table.

In [39]:
%sql SELECT COUNT(ID) AS Total_Num_Crimes FROM CHICAGO_CRIME_DATA

 * sqlite:///FinalDB.db
Done.


Total_Num_Crimes
533


### Problem 2
List community area names and numbers with per capita income less than 11000.

In [40]:
%sql SELECT DISTINCT(CD.COMMUNITY_AREA_NAME) \
FROM CENSUS_DATA CD, CHICAGO_CRIME_DATA CHC \
WHERE CD.COMMUNITY_AREA_NUMBER = CHC.COMMUNITY_AREA_NUMBER \
AND CD.PER_CAPITA_INCOME < 11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


### Problem 3
List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [41]:
%sql SELECT DISTINCT CASE_NUMBER, DESCRIPTION \
FROM CHICAGO_CRIME_DATA \
WHERE DESCRIPTION LIKE '%MINOR%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


### Problem 4
List all kidnapping crimes involving a child?

In [42]:
%sql SELECT DISTINCT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION \
FROM CHICAGO_CRIME_DATA \
WHERE PRIMARY_TYPE LIKE '%KIDNAP%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


### Problem 5
List the kind of crimes that were recorded at schools. (No repetitions)

In [45]:
%sql SELECT DISTINCT (PRIMARY_TYPE), DESCRIPTION, LOCATION_DESCRIPTION \
FROM CHICAGO_CRIME_DATA\
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION
BATTERY,SIMPLE,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING"
BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,TO VEHICLE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,POSS: HEROIN(WHITE),"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,MANU/DEL:CANNABIS 10GM OR LESS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,TO LAND,"SCHOOL, PUBLIC, GROUNDS"
PUBLIC PEACE VIOLATION,BOMB THREAT,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,BOMB THREAT,"SCHOOL, PUBLIC, BUILDING"


### Problem 6
List the type of schools along with the average safety score for each type.

In [55]:
%sql SELECT `Elementary, Middle, or High School`, ROUND(AVG(SAFETY_SCORE), 2) AS average_safety_score \
FROM CHICAGO_PUBLIC_SCHOOLS \
GROUP BY `Elementary, Middle, or High School`

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",average_safety_score
ES,49.52
HS,49.62
MS,48.0


### Problem 7
List 5 community areas with highest % of households below poverty line

In [58]:
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY \
FROM CENSUS_DATA\
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8
Which community area is most crime prone? Display the coumminty area number only.

In [62]:
%sql SELECT COMMUNITY_AREA_NUMBER, COUNT(COMMUNITY_AREA_NUMBER) AS FREQUENCY \
FROM CHICAGO_CRIME_DATA \
GROUP BY COMMUNITY_AREA_NUMBER\
ORDER BY FREQUENCY DESC LIMIT 1


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,FREQUENCY
25.0,43


### Problem 9
Use a sub-query to find the name of the community area with highest hardship index

In [69]:
%sql SELECT COMMUNITY_AREA_NAME \
FROM CENSUS_DATA \
WHERE HARDSHIP_INDEX IN (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA )


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


### Problem 10
Use a sub-query to determine the Community Area Name with most number of crimes?

In [87]:

%sql SELECT COMMUNITY_AREA_NAME \
FROM CENSUS_DATA CD \
WHERE COMMUNITY_AREA_NUMBER IN \
(SELECT COMMUNITY_AREA_NUMBER \
FROM CHICAGO_CRIME_DATA \
GROUP BY COMMUNITY_AREA_NUMBER \
ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC LIMIT 1)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
